

# Linear Model (Workbook)



<br/>
<br/><br/><br/>
### ITCS6156/8156 Spring 2018
### Minwoo "Jake" Lee

In [1]:
import numpy as np
import matplotlib.pyplot as plt 
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
#%matplotlib notebook
import pandas as pd

## Reading and applying linear models

In [ ]:
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/forest-fires/forestfires.csv -o forestfires.csv

In [ ]:
# Can I see the map image by using the map? 
df = pd.read_csv("forestfires.csv")

In [ ]:
x, y = np.meshgrid(np.arange(7), np.arange(10))
x

In [ ]:
y

In [ ]:
burned = df.loc[df.loc[:, 'area'] > 0, ['X', 'Y', 'area']]

In [ ]:
fig = plt.figure()
#ax = fig.add_subplot(111, projection='3d')

#ax.scatter(df.loc[:, 'X'], df.loc[:, 'Y'], df.loc[:, 'area'], marker='^')
xs, ys = np.meshgrid(range(10), range(10))
xs

zs = np.zeros(xs.shape)
for i, row  in burned.iterrows():
    x, y = int(row['X']), int(row['Y'])
    #print (x,y)
    #if zs[x, y] > 0:
    #    print(x, y, ":duplicate entries!")
    zs[x, y] += row['area']
    
plt.contourf(xs, ys, zs, alpha=0.7, cmap=plt.cm.inferno)
plt.grid(c='k', ls='-', alpha=0.3)

#plt.show()

In [ ]:
pd.DataFrame(zs)

In [ ]:
plt.contourf(xs, ys, np.log(zs + 1), alpha=0.7, cmap=plt.cm.inferno)

In [ ]:
burned.loc[:, ['X', 'Y']].hist()

In [ ]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
hist, xedges, yedges = np.histogram2d(burned.loc[:, 'X'], burned.loc[:, 'Y'], bins=9, range=[[1, 9], [1, 9]])

# Construct arrays for the anchor positions of the 16 bars.
# Note: np.meshgrid gives arrays in (ny, nx) so we use 'F' to flatten xpos,
# ypos in column-major order. For numpy >= 1.7, we could instead call meshgrid
# with indexing='ij'.
xpos, ypos = np.meshgrid(xedges[:-1] + 0.25, yedges[:-1] + 0.25)
xpos = xpos.flatten('F')
ypos = ypos.flatten('F')
zpos = np.zeros_like(xpos)

# Construct arrays with the dimensions for the 16 bars.
dx = 0.5 * np.ones_like(zpos)
dy = dx.copy()
dz = hist.flatten()

ax.bar3d(xpos, ypos, zpos, dx, dy, dz, color='b', zsort='average')

plt.show()

In [ ]:
X = df.iloc[:, :-1]
T = df.iloc[:, -1].as_matrix()
N = df.shape[0]

# forgot conversion of string
monthdic = {'jan': 1, 'feb':2, 'mar':3, 'apr':4, 'may':5, 'jun':6, 
            'jul': 7, 'aug':8, 'sep':9, 'oct':10, 'nov': 11, 'dec': 12}
daydic = {'mon': 1, 'tue':2, 'wed':3, 'thu':4, 'fri':5, 'sat':6, 'sun':7 }

X['month'] = X['month'].apply(lambda x: monthdic[x])
X['day'] = X['day'].apply(lambda x: daydic[x])
X = X.as_matrix()

Tlog = np.log(T + 1)

In [ ]:
# adding basis
X1 = np.hstack((np.ones((N,1)), X))
X1

In [ ]:
w = np.linalg.lstsq(X1.T @ X1, X1.T @ Tlog)[0]

In [ ]:
w

In [ ]:
Y = X1 @ w

In [ ]:
# RMSE 

E = Tlog-Y
E

In [ ]:
(Tlog - Y)**2

In [ ]:
np.mean((Tlog - Y)**2)

In [ ]:
np.sqrt(np.mean((Tlog - Y)**2))

In [ ]:
pd.DataFrame(T).describe()

In [ ]:
plt.plot(Tlog, 'ob')
plt.plot(Y, 'xr')

In [ ]:
w

# How to Improve the Fit? 

- Normalizaiton (Standardization) 

In [ ]:
pd.DataFrame(X).describe()

In [ ]:
mu = np.mean(X, axis=0)
std = np.std(X, axis=0)

In [ ]:
normX = (X - mu) / std
pd.DataFrame(normX).describe()

In [ ]:
dfX = pd.DataFrame(X)
mu = dfX.mean() 
std = dfX.std()

In [ ]:
ndfX = (dfX - mu) / std

In [ ]:
ndfX

In [ ]:
ndfX.describe()

In [ ]:
X1s = np.hstack((np.ones((N, 1)), ndfX.as_matrix())) 

In [ ]:
w_s = np.linalg.lstsq(X1s.T @ X1s, X1s.T @ Tlog)[0]

In [ ]:
Ys = X1s @ w_s

In [ ]:
plt.plot(Tlog, 'ob')
plt.plot(Ys, 'xr')

In [ ]:
w_s

In [ ]:
np.sqrt(np.sum((Tlog- Ys)**2))

# Does normalization help? 

- Normally
  - better results
  - more informative weights

- But, still the above results are poor! 
- what's next ?

In [ ]:
df.loc[:,'month']

In [ ]:
pd.get_dummies(df.loc[:, 'month'])

In [ ]:
pd.concat([pd.get_dummies(df.loc[:, 'month']), pd.get_dummies(df.loc[:, 'day'])], axis=1)

In [ ]:
df.columns.values

In [ ]:
X = pd.concat([
        df.iloc[:, :2], 
        pd.get_dummies(df.loc[:, 'month']), 
        pd.get_dummies(df.loc[:, 'day']),
        df.iloc[:, 4:-1]],
        axis=1)
X

In [ ]:
def normalize(X):
    mu = X.mean() 
    std = X.std()
    return (X - mu) / std

In [ ]:
nX = normalize(X)

In [ ]:
nX.shape

In [ ]:
nX.describe()

In [ ]:
X1i = np.hstack((np.ones((N, 1)), nX.as_matrix()))

In [ ]:
w_i = np.linalg.lstsq(X1i.T @ X1i, X1i.T @ Tlog)[0]

In [ ]:
Yi = X1i @ w_i

In [ ]:
np.sqrt(np.mean((Tlog- Yi)**2))

In [ ]:
plt.plot(Tlog)
plt.plot(Y)

# Helpful? What is next? 

- more flexible or sophistigated model!
- will comeback later in this semester. 

# Least Mean Squares 

Let us switch the gear and practice LMS, the online learning algorithm. 

In [ ]:
import IPython.display as ipd  # for display and clear_output


# read one by one and update weights 
alpha = 0.001 

w_lms = np.random.rand(X1i.shape[1])

fig = plt.figure(figsize=(16,8))

errs = []
for i in range(N):
    w_lms -= alpha * (w_lms.T @ X1i[i] - Tlog[i]) * X1i[i]
    
    Y_lms = X1i @ w_lms
    errs.append( np.sqrt(np.mean(Tlog - Y_lms)**2) )
    
    plt.clf()
    plt.subplot(1,2, 1)
    plt.plot(errs)
    plt.ylabel("RMSE")
    
    plt.subplot(1,2, 2)
    plt.plot(Tlog[:i])
    plt.plot(Y_lms[:i])
    plt.ylabel("Current Estimation")
    
    
    ipd.clear_output(wait=True)
    ipd.display(fig)
ipd.clear_output(wait=True)

In [ ]:
print("RMSE: {0}".format(errs[-1]))

# Partitioning Data 

- necessity to consider generalization of your model
- measure for the performance with unseen data


- training and testing partition

In [ ]:
# now partition the data 

""" partitioning data

    parameters
    -----------
    X        pd.DataFrame
             input data to partition
    T        pd.DataFrame
             target labels to partition
    raito    list
             list of ratios for partitions (should be summed to 1) 
             the number of return pairs are different
"""
def partition(X, T, ratio=[0.8, 0.2]): 
    
    assert(np.sum(ratio) == 1)
    
    # shuffle the data indices 
    idxs = np.random.permutation(X.index)
    
    # the number of samples 
    N = X.shape[0]
    
    Xs = []
    Ts = []
    i = 0  # first index to zero
    for k, r in enumerate(ratio):
        nrows = int(round(N * r))  # number of rows
        
        if k == len(ratio) -1:
            Xs.append(X.iloc[i:, :])
            Ts.append(T.iloc[i:, :])
        else:
            Xs.append(X.iloc[i:i+nrows, :])
            Ts.append(T.iloc[i:i+nrows, :])
        
        i += nrows
    
    return Xs, Ts



In [ ]:
Xlst, Tlst = partition(pd.DataFrame(X1i), pd.DataFrame(Tlog))

In [ ]:
Xlst[0].shape

In [ ]:
Xlst[1].shape

In [ ]:
Tlst[0].shape

In [ ]:
# read one by one and update weights 
alpha = 0.001 

Xtrain, Xtest = [xx.as_matrix() for xx in Xlst]
Ttrain, Ttest = [tt.as_matrix() for tt in Tlst]
 
w = np.random.rand(Xtrain.shape[1], Ttrain.shape[1])

fig = plt.figure(figsize=(16,8))

errs = []
for i in range(Xtrain.shape[0]):
    w -= alpha * ((w.T @ Xtrain[i] - Ttrain[i]) * Xtrain[i, None].T)
    
Yp = Xtest @ w
np.sqrt(np.mean((Yp -  Ttest)**2))

In [ ]:

plt.plot(Ttest)
plt.plot(Yp)

In [ ]:
# now with linear regression
w = np.linalg.lstsq(Xtrain.T @ Xtrain, Xtrain.T @ Ttrain)[0]

Yp2 = Xtest @ w
np.sqrt(np.mean((Yp2 -  Ttest)**2))

In [ ]:
plt.plot(Ttest)
plt.plot(Yp2)
